In [1]:
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc_old_segments")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
Using custom data configuration corpus-48c43d9b917d9e96
Reusing dataset text (/Users/Gianni/.cache/huggingface/datasets/text/corpus-48c43d9b917d9e96/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 3/3 [00:00<00:00, 36.01it/s]
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further trainin

In [3]:
import tensorflow as tf

In [45]:
sent = semiotic.corpus.train["text"][10]
# sent = "Whenever it is possible bring your own lunch to the office"
sent

'intime thefuture and vi abilityof rural communities maybe threatened standing conference of rural community councils 1978'

In [46]:
sent_len = len(sent.split())
sent_mask = [" ".join([t if i!=n else "[MASK]" for i,t in enumerate(sent.split())]) for n in range(sent_len)]

In [73]:
input = semiotic.paradigmatic.bert_tokenizer(sent_mask)
outputs = semiotic.paradigmatic.model(input["input_ids"])

# input = semiotic.paradigmatic.hf_tokenizer(sent_mask, return_tensors="tf")
# outputs = semiotic.paradigmatic.hf_model(input)
# Gather all the masked logits in a batch, by taking the row diagonal of the 3D tensor

parad_logits = tf.gather_nd(outputs.logits, indices=[[i,i] for i in range(sent_len)])
# parad_logits = tf.gather_nd(outputs.logits[:,1:-1], indices=[[i,i] for i in range(sent_len)])

In [74]:
parad_logits

<tf.Tensor: shape=(16, 9872), dtype=float32, numpy=
array([[-11.827877 ,  -6.459899 , -11.9156275, ...,  -4.435776 ,
         -6.784652 ,  -4.5387135],
       [-11.768407 ,  -6.480848 , -11.818177 , ...,  -4.3940487,
         -6.729151 ,  -4.410543 ],
       [-11.777363 ,  -6.4691577, -11.900703 , ...,  -4.3257294,
         -6.724357 ,  -4.503372 ],
       ...,
       [-11.458163 ,  -6.439986 , -11.573915 , ...,  -4.1614428,
         -6.587628 ,  -4.4059668],
       [-11.589834 ,  -6.4382563, -11.684255 , ...,  -4.338984 ,
         -6.628087 ,  -4.4184666],
       [-11.593312 ,  -6.4351892, -11.664438 , ...,  -4.558982 ,
         -6.617267 ,  -4.5149183]], dtype=float32)>

In [77]:
input = semiotic.paradigmatic.bert_tokenizer(sent)
outputs = semiotic.paradigmatic.model(input["input_ids"])
parad_logits = outputs.logits[0]

In [86]:
input["input_ids"].numpy()

array([[4896, 4432,    5,  846, 3119, 4427, 5027, 1046,    1,    1,    1,
           8, 4427, 1078,    1, 8492]], dtype=int32)

In [88]:
semiotic.paradigmatic.tokenizer.decode(input["input_ids"][0])

'intime thefuture and vi abilityof rural communities maybe [UNK] [UNK] [UNK] of rural community [UNK] 1978'

In [79]:
logits_positif = tf.nn.relu(parad_logits)
probs, norms = tf.linalg.normalize(logits_positif, ord=1, axis=1)

# Maybe its cheaper to compute top k for k = non_zero, which would differ from row to row, and hence not yeld a tensor. To be tested.

non_zeroes = tf.math.count_nonzero(probs,axis=1)
max_non_zeroes = max(non_zeroes).numpy()

parad_data = tf.math.top_k(probs, k = max_non_zeroes, sorted=True)
parad_tokens = [semiotic.syntagmatic.tokenizer.decode(parad.numpy()[:parad_len]).split() for parad, parad_len in zip(parad_data.indices, non_zeroes)]

# parad_tokens = [semiotic.paradigmatic.hf_tokenizer.decode(parad.numpy()[:parad_len]).split() for parad, parad_len in zip(parad_data.indices, non_zeroes)]
print(sent)
slg.util.df([p[:20] for p in parad_tokens], keys = sent.split())

intime thefuture and vi abilityof rural communities maybe threatened standing conference of rural community councils 1978


,intime,thefuture,and,vi,abilityof,rural,communities,maybe,threatened,standing,conference,of,rural,community,councils,1978
0,intime,thefuture,and,vi,abilityof,rural,communities,maybe,and,and,and,of,rural,community,and,1978
1,but,thenature,sand,vari,ability,urban,hospitals,canbe,that,that,that,ofthe,urban,economy,that,1984
2,again,theworld,or,cap,able,industrial,countries,willbe,of,of,of,sof,jewish,union,of,1987
3,further,thesocial,andthe,predict,mentof,local,firms,havebeen,but,but,but,ofour,social,education,but,1988
4,thus,thebusiness,andre,dis,anceof,social,britain,mightbe,which,which,which,in,northern,university,which,1981
5,clearly,growth,ands,accept,ableto,regional,centres,couldbe,in,in,in,from,industrial,development,in,1980
6,and,thewhole,andhis,in,ably,jewish,france,scanbe,s,s,s,for,regional,family,s,etal
7,atthesametime,theco,edand,toler,ingof,northern,population,wouldbe,about,about,about,on,residential,industry,about,1975
8,however,themass,eand,suit,abilityto,residential,psychology,needtobe,or,or,or,ofits,local,business,or,1970
9,these,aun,yand,enjoy,mentofthe,human,politicians,mustbe,all,all,all,s,modern,state,all,1986


In [55]:
max_parad = 100 # max_non_zeroes
slg.util.plot_scatter_line(list(range(max_parad)),parad_data.values[0],sent.split()[0], add_trace=[(list(range(max_parad)), v, t) for v, t in zip(parad_data.values[1:],sent.split()[1:])])

In [56]:
non_zeroes

<tf.Tensor: shape=(16,), dtype=int64, numpy=
array([ 224, 1701,  231,  919, 1084, 2907, 5993,  915,  225,  225,  225,
        284, 3342,  575,  225, 1182])>